In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
os.chdir('/content/drive/My Drive/DSC650/assignment02')
!pwd

Mounted at /content/drive
/content/drive/My Drive/DSC650/assignment02


In [3]:
!pip install tinydb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


class DocumentDB(object):
    people_json = kv_data_dir.joinpath('people.json')
    visited_json = kv_data_dir.joinpath('visited.json')
    sites_json = kv_data_dir.joinpath('sites.json')
    measurements_json = kv_data_dir.joinpath('measurements.json')

    # open json files
    with open(sites_json) as f:
        _sites_data = json.load(f)

    with open(measurements_json) as f:
        _measurements_data = json.load(f)

    with open(people_json) as f:
        _people_data = json.load(f)

    with open(visited_json) as f:
        _visited_data = json.load(f)

    
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = None
        self._load_db()

    
    def _get_sites(self, site_id):
        '''
        Get site

        args:
          site_id (str)

        returns:
          site (json)
        '''
        site = self._sites_data[str(site_id)]
        return site

    
    def _get_measurements(self, person_id):
        '''
        Get measurements

        args:
          person_id (str)

        returns:
          measurements (json)
        '''
        measurements = []

        # for all measurements
        # if correct measurement person_id add measurement to array
        for measurement in self._measurements_data.values():
            if str(measurement['person_id']) == str(person_id):
                measurements.extend([measurement])
        
        return measurements


    def _get_visits(self, visit_id):
        '''
        Get visits and sites

        args:
          visit_id (str)

        returns:
          visit (array)
        '''
        visit = [visit for key, visit in self._visited_data.items() if visit['visit_id'] == visit_id][0]
        site_id = visit['site_id']

        site = self._get_sites(site_id)
        
        visit['site'] = site

        return visit


    def _load_db(self):
        self._db = TinyDB(self._db_path)

        people = self._people_data.items()
        
        for person_id, person_data in people:
            measurements = self._get_measurements(person_id)
            visit_ids = set([measurement['visit_id'] for measurement in measurements])
            
            visits = []
            for visit_id in visit_ids:
                visit = self._get_visits(visit_id)

                visit['measurements'] = [measurement for measurement in measurements if visit_id == measurement['visit_id']]

                visits.append(visit)

            person_data['visits'] = visits
            #print(json.dumps(person_data, indent = 4))
            self._db.insert(person_data)           

In [5]:
db_path = results_dir.joinpath('patient-info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)